## **Task 3: Simulation Controller - Sedra Alkahef**

This task is responsible for managing the entire simulation process, by developing a **`Simulation`** class and a **`SimulationGUI`** class.

**The objectives and functionalities of this task are the following:** 
- Initializes particles with random positions, velocities, and particle types.
- Control the simulation loop using adjustable time steps, updating particle states over time.
- Allows users to start, pause, resume, and restart the simulation.
- Implements a GUI that provides user control over simulation parameters (particle count, speed, energy, particle type, and time step).

**The **`Simulation`** class is created with the following attributes:**

- **`num_particles`**: number of particles in the simulation.
- **`speed`**: maximum initial velocity of particles.
- **`time_step`**: time interval for simulation updates.
- **`max_time`**: simulation duration.
- **`particle_type`**: type of colliding particles in the simulation (e.g., proton, electron).
- **`energy_ev`**: energy of particles in electron volts (eV).<br>
This class also includes internal states such as a particle list, current time, and a running status flag to track whether the simulation is active.

**Key Functionalities of the **`Simulation`** Class:**

- It has a method for Initializing Particles:<br>
Particles are initialized with random positions, velocities, and masses.

- It has methods for Controlling the Simulation Loop:<br>
o The simulation progresses in discrete time steps. At each step, the particle positions are updated using their velocity.<br>
o Detects collisions via the integrated Collider class from Task 2.<br>
o Handles particle decay and generates decay products as defined in Task 1.<br>

- It has a method allowing Start, Pause, Resume, and Reset:<br>
o The simulation can be started, paused, resumed, or reset by toggling its internal running state.<br>
o It uses Python's **`threading`** library to ensure that the simulation runs at the same time as the GUI, preventing interface freezing.<br>


The **`SimulationGUI`** Class:<br>
Creates a Graphical User Interface (GUI) for controlling the simulation, using the **`tkinter`** library.<br>

- Provides user input fields and dropdown menus allowing the user to set the particle parameters of the simulation such as: (Number of particles, speed, particle type, energy in eV, and time-step)
- Buttons allow users to start, pause, resume, and reset the simulation.
- A status update displays the current status of the simulation (e.g., "Running", "Paused", "Not Started").
- Includes Error Handling for wrong user input:<br>
o Validates user inputs to ensure correct data types (e.g., numerical values for speed, particle count, energy, and time step).<br>
o Uses **`tkinter.messagebox`** to alert users of invalid inputs.<br>

**Object-Oriented Programming (OOP) Principles:**<br>
- Encapsulation:<br>
The **`Simulation`** class encapsulates all attributes and methods required to manage the simulation. The **`SimulationGUI`** class encapsulates the GUI's logic, handes user inputs and controls the simulation using start, pause, resume, and restart buttons.<br>

- Abstraction:<br>
The **`Simulation`** class abstracts the complexities of particle initialization, position updates, and decay detection.<br>
The **`SimulationGUI`** class provides a user-friendly interface for controlling the simulation, abstracting the simulation logic.<br>

- Inheritance:<br>
This task doesn't define new subclasses, but it relies on inherited behavior from the **`Particle`** and **`Collider`** classes (from Tasks 1 and 2).


In [ ]:
import numpy as np
import tkinter as tk
from tkinter import messagebox
import threading
import random

class Simulation:
    def __init__(self, num_particles=10, speed=1000000, time_step=0.001, max_time=10, particle_type="proton", energy_ev=1e13):
        self.num_particles = num_particles
        self.speed = speed
        self.time_step = time_step
        self.max_time = max_time
        self.particle_type = particle_type
        self.energy_ev = energy_ev
        self.energy_joules = self.convert_ev_to_joules(energy_ev)
        self.particles = []  
        self.time = 0
        self.running = False
        self.simulation_thread = None
        self.create_particles()

    def convert_ev_to_joules(self, energy_ev):
        return energy_ev * 1.60218e-19

    def create_particles(self):
        """ Initializes particles with random positions, velocities, masses, and particle types """
        self.particles = []
        for _ in range(self.num_particles):
            position = np.random.rand(3) * 100  # Random positions
            velocity = np.random.rand(3) * self.speed  # Random velocity
            mass = random.uniform(1e-27, 1e-24)  # Random mass for particles
            self.particles.append(Particle(position, velocity, mass, self.particle_type))

    def update(self):
        """ Updates the simulation, checks for collisions, particle movement, and decay events """
        if self.running:
            self.time += self.time_step
            collider = Collider(self.particles)
            collider.detect_collisions()
            for particle in self.particles:
                particle.update_position(self.time_step)
                decay_products = particle.detect_decay()
                if decay_products:
                    self.particles.extend(decay_products)

    def start(self):
        """ Starts the simulation in a new thread """
        self.running = True
        self.simulation_thread = threading.Thread(target=self.run_simulation)
        self.simulation_thread.start()

    def pause(self):
        """ Pauses the simulation """
        self.running = False

    def reset(self):
        """ Resets the simulation to its initial state """
        self.time = 0
        self.running = False
        self.create_particles()

    def resume(self):
        """ Resumes the simulation """
        if not self.running:
            self.running = True
            self.simulation_thread = threading.Thread(target=self.run_simulation)
            self.simulation_thread.start()

    def run_simulation(self):
        """ Runs the simulation loop until the maximum time duration is reached or the simulation is stopped """
        while self.running and self.time < self.max_time:
            self.update()


# GUI for controlling the simulation
class SimulationGUI:
    def __init__(self, root, simulation):
        self.root = root
        self.simulation = simulation
        self.root.title("Particle Collider Simulation")
        self.root.geometry("400x500")  

        # Main frame for input controls
        self.form_frame = tk.Frame(root)
        self.form_frame.pack(padx=10, pady=10)

        # Number of particles input field
        self.num_particles_label = tk.Label(self.form_frame, text="Number of Particles:")
        self.num_particles_label.grid(row=0, column=0, pady=5)
        self.num_particles_entry = tk.Entry(self.form_frame, width=30)
        self.num_particles_entry.insert(0, "10")  # Default number of colliding particles 
        self.num_particles_entry.grid(row=0, column=1, pady=5)

        # Particle speed input field
        self.speed_label = tk.Label(self.form_frame, text="Particle Speed (m/s):")
        self.speed_label.grid(row=1, column=0, pady=5)
        self.speed_entry = tk.Entry(self.form_frame, width=30)
        self.speed_entry.insert(0, "1000000")  # Default speed
        self.speed_entry.grid(row=1, column=1, pady=5)

        # Time step input field
        self.time_step_label = tk.Label(self.form_frame, text="Time Step (seconds):")
        self.time_step_label.grid(row=2, column=0, pady=5)
        self.time_step_entry = tk.Entry(self.form_frame, width=30)
        self.time_step_entry.insert(0, "0.001")  # Default time step
        self.time_step_entry.grid(row=2, column=1, pady=5)

        # Particle type dropdown menu
        self.particle_type_label = tk.Label(self.form_frame, text="Select Particle Type:")
        self.particle_type_label.grid(row=3, column=0, pady=5)
        self.particle_type_var = tk.StringVar()
        self.particle_type_var.set("proton")  # Default particle type
        self.particle_type_menu = tk.OptionMenu(self.form_frame, self.particle_type_var, "proton", "electron", "neutron")
        self.particle_type_menu.config(width=20)
        self.particle_type_menu.grid(row=3, column=1, pady=5)

        # Energy of colliding particles input field
        self.energy_label = tk.Label(self.form_frame, text="Energy of Colliding Particles (eV):")
        self.energy_label.grid(row=4, column=0, pady=5)
        self.energy_entry = tk.Entry(self.form_frame, width=30)
        self.energy_entry.insert(0, "1e13")  # Default energy
        self.energy_entry.grid(row=4, column=1, pady=5)

        self.status_label = tk.Label(self.form_frame, text="Status: Not Started", width=40, height=2, relief="sunken")
        self.status_label.grid(row=5, column=0, columnspan=2, pady=10)

        # Control buttons
        self.start_button = tk.Button(self.form_frame, text="Start", width=20, command=self.start_simulation)
        self.start_button.grid(row=6, column=0, columnspan=2, pady=10)

        self.pause_button = tk.Button(self.form_frame, text="Pause", width=20, command=self.pause_simulation)
        self.pause_button.grid(row=7, column=0, columnspan=2, pady=5)

        self.reset_button = tk.Button(self.form_frame, text="Reset", width=20, command=self.reset_simulation)
        self.reset_button.grid(row=8, column=0, columnspan=2, pady=5)

        self.resume_button = tk.Button(self.form_frame, text="Resume", width=20, command=self.resume_simulation)
        self.resume_button.grid(row=9, column=0, columnspan=2, pady=5)

    def start_simulation(self):
        """ Starts the simulation """
        try:
            num_particles = int(self.num_particles_entry.get())
            speed = float(self.speed_entry.get())
            time_step = float(self.time_step_entry.get())
            particle_type = self.particle_type_var.get()
            energy_ev = float(self.energy_entry.get())
            self.simulation.num_particles = num_particles
            self.simulation.speed = speed
            self.simulation.time_step = time_step
            self.simulation.particle_type = particle_type
            self.simulation.energy_ev = energy_ev
            self.simulation.energy_joules = self.simulation.convert_ev_to_joules(energy_ev)
            self.simulation.create_particles()
            self.simulation.start()

            # Update status label
            self.status_label.config(text="Simulation Status: Running")
        except ValueError:
            messagebox.showerror("Input Error", "Please enter valid numerical values.")

    def pause_simulation(self):
        """ Pauses the simulation. """
        self.simulation.pause()
        self.status_label.config(text="Simulation Status: Paused")

    def reset_simulation(self):
        """ Resets the simulation """
        self.simulation.reset()
        self.status_label.config(text="Simulation Status: Not Started")

    def resume_simulation(self):
        """ Resumes the simulation """
        self.simulation.resume()
        self.status_label.config(text="Simulation Status: Running")


# Main function to set up and run the GUI
def main():
    root = tk.Tk()
    simulation = Simulation()
    gui = SimulationGUI(root, simulation)
    root.mainloop()

if __name__ == "__main__":
    main()
